In [1]:
import os
import glob
from tqdm.notebook import tqdm
import networkx as nx
import time
import pandas as pd

In [2]:
os.chdir(r'C:\Users\Kislee\PycharmProjects\Korean_textbook')
DATA_PATH = os.getcwd() + r'\target_data'
print(DATA_PATH)

C:\Users\Kislee\PycharmProjects\Korean_textbook\target_data


In [3]:
files = glob.glob(os.path.join(DATA_PATH, '*.xlsx'))

In [4]:
all_dfs = []
for source in files:
    temp_df = pd.read_excel(source, engine='openpyxl')
    all_dfs.append(temp_df)
df = pd.concat(all_dfs)

In [5]:
df = df[df['등급'].isin([1,2,3,4,5,6])]

In [6]:
len(df)

557

In [7]:
df.head()

,학교명,등급,지문
0,경희대,1,1) 가: 저는 장민입니다.\n 나: 저는 한나 요한손입니다.\n2) 가:...
1,경희대,1,1) 가: 어느 나라 사람입니까\n 나: 저는 러시아 사람입니다.\n2) 가: ...
2,경희대,1,1) 제 친구는 친절해요.\n2) 이 사람은 제 친구 한나예요. 한나는 정말 똑똑해...
3,경희대,1,1) 가: 그게 뭐예요?\n 나: 커피예요.\n2) 가: 이게 한국어로 뭐예요?...
4,경희대,1,1) 가: 서점이 어디에 있어요?\n 나: 2층에 있어요.\n2) 가: 커피숍...


In [8]:
cell_df = df['지문']

In [9]:
#문장 분리 : . ! ? 를 이용해서 간단히 처리

In [10]:
cell_df = [ cell.replace('\n','') for cell in cell_df]
cell_df = [ cell.replace('.','.$') for cell in cell_df]
cell_df = [ cell.replace('!','!$') for cell in cell_df]
cell_df = [ cell.replace('?','?$') for cell in cell_df]

In [11]:
sentences_in_cell = [cell.split('$') for cell in cell_df]

In [12]:
all_sentences = []
for cell in sentences_in_cell:
    for sent in cell:
        if len(sent) > 2:
            all_sentences.append(sent.strip())

In [13]:
#UTagger
from Utagger.bin.utagger_py import UTagger

In [14]:
os.chdir(r'C:\Users\Kislee\PycharmProjects\Korean_textbook\Utagger\한국어교재지문')
print(os.getcwd())

C:\Users\Kislee\PycharmProjects\Korean_textbook\Utagger\한국어교재지문


In [15]:
dllpath = r'../bin/UTaggerR64.dll'
cfgpath = r'../Hlxcfg.txt'

rt = UTagger.Load_global(dllpath, cfgpath)
ut = UTagger(0) # 0은 객체 고유번호. 0~99 지정 가능. 같은 번호로 여러번 생성하면 안됨. 한 스레드당 하나씩 지정 필요.
rt = ut.new_ucma() #객체 생성. 객체가 있어야 유태거 이용 가능.

python call utagger function
C:\Users\Kislee\PycharmProjects\Korean_textbook\Utagger\Hlxcfg.txt


In [16]:
sent_utagging = [ut.tag_line(sent,3) for sent in tqdm(all_sentences)]

  0%|          | 0/33988 [00:00<?, ?it/s]

In [17]:
ut.release_ucma() #객체 해제
UTagger.Release_global() #사전 해제

In [18]:
#단어 어깨번호 정제하고자 할 때 사용, 어깨번호는 여러숫자로 구성될 수 있으므로 마지막 2개만 사용
def cutSemanticNum(word, all=False): # all : True이면 의미숫자를 다 잘라버림, False이면 두글자만 남김
    splitted = word.split('__')
    
    if len(splitted) == 1: # __ 구분자가 없는단어는 원 단어 그대로 반환
        return word
    
    if all==True:
        return splitted[0]
    
    out_word = splitted[0]+'__'+splitted[1][-2:]
    return out_word    

In [19]:
# utagger 가 생성한 한 어절에서 + 단위로 형태소를 자른 후 / 단위로 어휘와 태그를 분석
def utagging_to_simple_tagging(utg):
    #outlier가 발견되어 제거함
    utg = utg.replace('NNG\n','NNG')
    
    temp_wordset = utg.split('+') # utagger 가 생성한 결과에서 + 단위로 잘라줌
   
    wordset = []
    for word_tag in temp_wordset:
        seplist = word_tag.split('/') #/ 로 잘라지지 않는 경우에 대한 예외처리
        if len(seplist) <= 1 :
            continue
        else :
            wordset.append(seplist)
    if len(wordset) <= 0:
        return None
   
    try:
        first_word = wordset[0][0]
        first_tag = wordset[0][1]
    except:
        print("예외처리 : ", wordset)
    
    #동사(VV), 형용사(VA)로 시작하면 바로 첫 단어만 내보냄
    if first_tag.startswith('VV') or first_tag.startswith('VA'):
        return (first_word, first_tag)
    
    if len(wordset) == 1: #한 단어인 경우 
         #명사류, 부사, 한글자 이상의 외국어는 사용
        if first_tag.startswith('NNP') or first_tag.startswith('NNG') or first_tag.startswith('MAG') or (first_tag.startswith('SL') and len(first_word)>1) :
            #if word_tag[1].startswith('SL'):
            #            print(word_tag[0])
            return (first_word, first_tag)

    else: #여러 단어인 경우 
        if first_tag.startswith('NNP') or first_tag.startswith('NNG') or first_tag.startswith('SL') or first_tag.startswith('XPN'): #첫 단어가 다음과 같으면
            output = []
            
            if first_tag.startswith('XPN'): #XPN+NNG 는 합쳐서 NNG로
                second_word = wordset[1][0]
                second_tag = wordset[1][1]
                if second_tag.startswith('NNG'):
                        output.append((cutSemanticNum(first_word,True)+second_word, second_tag))
                        #print(output[0])
                return output
            
            for word_tag in wordset: #XPN+NNG 로 반환해야함--> NNG로
                if word_tag[1].startswith('NNP') or word_tag[1].startswith('NNG') or (word_tag[1].startswith('SL') and len(word_tag[0])>1): #명사류를 찾음
                    #if word_tag[1].startswith('SL'): #지울 것
                    #    print(word_tag[0])
                    output.append((word_tag[0], word_tag[1]))
            return output

    return None

#문장 전체를 받아서 (단어,태그) 튜플 리스트로 반환
def remain_available_words(doc): 
    result = []
    for utg in doc.split(' '):
        simple_tagged = utagging_to_simple_tagging(utg)
        if simple_tagged == None:
            continue
                
        if str(type(simple_tagged)) =="<class 'tuple'>":
            result.append(simple_tagged)
        else :
            for word_tag in simple_tagged:
                result.append(word_tag)
    return result

In [20]:
sent_tagging = [remain_available_words(sent) for sent in sent_utagging]

In [21]:
#불용어 수집
os.chdir(r'C:\Users\Kislee\PycharmProjects\Korean_textbook')
DATA_PATH = os.getcwd()
filename = DATA_PATH+r'\불용어목록.xlsx'
stopword_df = pd.read_excel(filename, engine='openpyxl')

In [22]:
#불용어 제거 필터
stopword = list(stopword_df['불용어'])
def stopword_filter(cell):
    output = []
    for word, tag in cell:
        if word not in stopword:
            output.append((word,tag))
    return output      

In [23]:
#불용어 제거
sent_tagging = [stopword_filter(cell) for cell in sent_tagging]

In [24]:
#태그:빈도 사전구성
tag_freq_dic = {}
for cell in sent_tagging:
    for word, tag in cell:
        tag_freq_dic[tag] = tag_freq_dic.get(tag, 0) + 1
print("고유 태그 수 : %d" % len(tag_freq_dic))

고유 태그 수 : 6


In [25]:
#단어:태그 사전구성
word_tag_dic = {}
for sent in sent_tagging:
    for word, tag in sent:
        word_tag_dic[word] = tag

In [26]:
#셀내의 튜플 리스트에서 워드만 추출
def word_selector(cell):
    output = []
    for word, tag in cell:
        if word not in stopword:
            output.append(word)
    return output      

In [27]:
sent_wordlist = [word_selector(cell) for cell in sent_tagging]

In [28]:
#EDA
print("문장 수 : ", len(sent_wordlist))

문장 수 :  33988


In [29]:
cnt = 0
for cell in sent_wordlist:
    cnt += len(cell)
print('단어 출현 수 :', cnt)

단어 출현 수 : 187301


In [30]:
#단어:빈도 사전
word_freq_dic = {}
for cell in sent_wordlist:
    for word in cell:
        word_freq_dic[word] = word_freq_dic.get(word, 0) + 1
print("고유 단어 수 : %d" % len(word_freq_dic))

고유 단어 수 : 13672


In [31]:
word_freq = []
for key, value in word_freq_dic.items():
    word_freq.append((value, key))
word_freq.sort(reverse=True)
print("상위단어 정렬", word_freq[:1000])

상위단어 정렬 [(5940, '있__01'), (2567, '보__01'), (1991, '사람'), (1915, '가__01'), (1672, '좋__01'), (1362, '같'), (1213, '없__01'), (1198, '한국__05'), (1157, '남자__02'), (1112, '여자__02'), (1100, '많'), (1049, '오__01'), (986, '먹__02'), (967, '친구__02'), (947, '많이'), (877, '잘__02'), (818, '일__01'), (800, '안__02'), (793, '남__02'), (751, '오늘'), (740, '알'), (739, '시간__04'), (704, '생각하'), (699, '좀__02'), (682, '더__01'), (665, '만들'), (630, '살__01'), (627, '지금__03'), (598, '정말__01'), (587, '듣__01'), (577, '말__01'), (569, '받__01'), (557, '요즘'), (537, '음식'), (526, '크__01'), (525, '집__01'), (506, '주__01'), (499, '학생'), (482, '좋아하'), (474, '같이'), (464, '이번__01'), (455, '안녕하'), (422, '생각__01'), (416, '만나'), (415, '문제__06'), (408, '들__01'), (399, '모르'), (398, '다음__01'), (392, '맞__01'), (391, '나오'), (380, '전__08'), (364, '영화__01'), (356, '너무__01'), (348, '다__03'), (344, '배우__01'), (342, '가지'), (337, '쓰__03'), (336, '학교'), (333, '사'), (330, '선생님'), (328, '힘들'), (323, '또'), (322, '회사__04'), (319, '자신__01'), (319, '이야

In [32]:
# 단어 붙이기
cell_wordjoin = [' '.join(cutSemanticNum(word) for word in cell) for cell in sent_wordlist]

In [33]:
#국가 목록
country_list = ['중국','일본','미국','베트남','프랑스','영국','태국','러시아','호주','인도','독일','스페인','노르웨이','이탈리아','그리스','필리핀','몽골', '캐나다','브라질','콜롬비아','케냐','칠레','이집트','미얀마','투발루','이란','스웨덴','몰디브','라오스','파라과이','코스타리카','자메이카','예멘','아이티','스위스','부탄','말레이시아','네팔','폴란드','파나마','터키','탄자니아','키리바시','캄보디아','인도네시아','우즈베키스탄','에티오피아','바레인','멕시코','나이지리아','과테말라']

In [34]:
len(country_list)

51

In [35]:
#입력 리스트에 대한 쌍 조합 튜플을 반환
def get_pairs_from_list(lst):
    pairs = []
    unique = list(set(lst)) #리스트 내 중복 제거
    lst_len = len(unique)
    for i in range(lst_len):
        if i==(lst_len-1):
            break
        subset = unique[i+1:]

        for item in subset:
            pairs.append((unique[i], item))
    return pairs

In [36]:
#국가명으로 연관단어 네트워크 분석
def country_semantic_network(country):
    #대상 셀 선정
    target_cell_wordjoin = [cell for cell in cell_wordjoin if country in cell]
    #단어 합치기
    target_cell_wordlist = [cell.split(" ") for cell in target_cell_wordjoin]
    word_dic = {}
    for cell in target_cell_wordlist:
        for word in cell:
            word_dic[word] = word_dic.get(word, 0) + 1
    print("총 단어 수 : %d" % len(word_dic))
    word_freq = []
    for key, value in word_dic.items():
        word_freq.append((value, key))
    word_freq.sort(reverse=True)
    #네트워크 생성, gephi를 위한 source-target 생성
    edges = [get_pairs_from_list(cell) for cell in target_cell_wordlist]
    G = nx.Graph()
    for comb in edges:
        for source, dest in comb:
            if G.has_edge(source, dest):
                G[source][dest]['weight'] += 1
            else:
                G.add_edge(source, dest, weight=1)
    print("생성된 그래프 노드 수: %d, 엣지 수 : %d" %(G.number_of_nodes(), G.number_of_edges()))
    ####degree centrality 추출 및 소팅(시간 소요됨)
    start = time.time()
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    print("네트워크 분석 완료 time : %.2f seconds" % (time.time()-start))
    #####degree 추출 및 소팅
    degree = G.degree()
    degree_freq = []
    for key, value in degree:
        degree_freq.append((value, key))
    degree_freq.sort(reverse=True)
    #print("상위 degree", degree_freq[:100])
    
    #결과 데이터 생성
    global country_df
    for dfreq, word in degree_freq[:200] :
        country_df = country_df.append({'품사':word_tag_dic[word],'단어':word,'출현빈도':word_dic[word],'연결단어수':dfreq,'연결중심성':degree_centrality[word],'매개중심성':betweenness_centrality[word],'분석범위':country,'출현빈도/품사빈도':word_freq_dic[word]/tag_freq_dic[word_tag_dic[word]]}, ignore_index=True)

In [37]:
#결과 데이터 생성
country_df = pd.DataFrame()
country_df = pd.DataFrame(columns=['품사','단어','출현빈도','출현빈도/품사빈도','연결단어수','연결중심성','매개중심성','분석범위'])

for country in country_list:
    print("================== %s 분석 시작 =====================" % country)
    country_semantic_network(country)
    print("================== %s 분석 종료 =====================" % country)
    print()
    
DATA_PATH = os.getcwd() + r'\save_data'
filename = DATA_PATH+'\국가별 연관단어 분석.xlsx'
country_df.to_excel(filename)
print("%s 저장 완료!" %filename)

================== 중국 분석 시작 =====================
총 단어 수 : 644
생성된 그래프 노드 수: 644, 엣지 수 : 5382
네트워크 분석 완료 time : 1.65 seconds
================== 중국 분석 종료 =====================

================== 일본 분석 시작 =====================
총 단어 수 : 501
생성된 그래프 노드 수: 501, 엣지 수 : 3959
네트워크 분석 완료 time : 0.92 seconds
================== 일본 분석 종료 =====================

================== 미국 분석 시작 =====================
총 단어 수 : 479
생성된 그래프 노드 수: 479, 엣지 수 : 3678
네트워크 분석 완료 time : 0.77 seconds
================== 미국 분석 종료 =====================

================== 베트남 분석 시작 =====================
총 단어 수 : 136
생성된 그래프 노드 수: 136, 엣지 수 : 554
네트워크 분석 완료 time : 0.06 seconds
================== 베트남 분석 종료 =====================

================== 프랑스 분석 시작 =====================
총 단어 수 : 162
생성된 그래프 노드 수: 162, 엣지 수 : 881
네트워크 분석 완료 time : 0.08 seconds
================== 프랑스 분석 종료 =====================

================== 영국 분석 시작 =====================
총 단어 수 : 156
생성된 그래프 노드 수: 156, 엣지 수 : 1092
네트워크 분석 완료 time : 0.08 s

================== 과테말라 분석 종료 =====================

C:\Users\Kislee\PycharmProjects\Korean_textbook\save_data\국가별 연관단어 분석.xlsx 저장 완료!


In [ ]:
#코드 끝.